# **NEXT WORD PREDICTION**

In [2]:
!pip install pandas numpy matplotlib nltk tensorflow

/bin/bash: line 1: /media/mjb/3E0B423C11D87515/disco_rigido/documentos/proyectos_programacion/Kaggle_projects/next_word_prediction/env/bin/pip: cannot execute: required file not found


## Librerias a usar

In [3]:
import pandas as pd
import numpy as np
import heapq
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras.layers import LSTM
import json
from tensorflow.keras.optimizers import RMSprop

2025-10-17 08:16:05.484794: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-17 08:16:05.523930: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-17 08:16:06.520567: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


## Abrimos el json que contiene nuestros datos


In [4]:
with open ('data/train.json', 'r') as f:
    data = json.load(f)

df = pd.DataFrame.from_dict(data)

df

,id,tag,title,original_language,conversation
0,190329_J07_03,phone call,伝言への折り返し電話打ち合わせ日程調整,ja,"[{'no': 1, 'ja_speaker': '土井さん', 'en_speaker':..."
1,190329_J06_11,meeting,株主総会に関する役員向け説明（２）,ja,"[{'no': 1, 'ja_speaker': '高田さん', 'en_speaker':..."
2,190329_J06_10,meeting,株主総会に関する役員向け説明（１）,ja,"[{'no': 1, 'ja_speaker': '友井さん', 'en_speaker':..."
3,190329_J06_09,face-to-face conversation,信託銀行から株主名簿を受領,ja,"[{'no': 1, 'ja_speaker': '高田さん', 'en_speaker':..."
4,190329_J06_08,face-to-face conversation,株主総会への臨場要請,ja,"[{'no': 1, 'ja_speaker': '高田さん', 'en_speaker':..."
...,...,...,...,...,...
665,190315_E001_05,presentation,Presentation: Company introduction,en,"[{'no': 1, 'en_speaker': 'Mr. Asano Yuki', 'ja..."
666,190315_E001_04,training,Training: training a new employee,en,"[{'no': 1, 'en_speaker': 'Mr. Ben Sherman', 'j..."
667,190315_E001_03,general chatting,General chatting: Company Christmas party,en,"[{'no': 1, 'en_speaker': 'Ms. Amy Lin', 'ja_sp..."
668,190315_E001_02,phone call,Phone call: requesting a lunch meeting.,en,"[{'no': 1, 'en_speaker': 'Mr. John Smith', 'ja..."


## En la columna conversation se puede notar rapidamente que hay una estructura anidada, en este paso quiero ver una muestra anidada


In [9]:
muestra_anidada = df['conversation'][12]
muestra_anidada[0:3]  # Primeras tres interacciones de la conversación 12

[{'no': 1,
  'ja_speaker': '岬さん',
  'en_speaker': 'Misaki-san',
  'ja_sentence': 'さて、本日のミーティングでは、みなさんに良い報告があります。',
  'en_sentence': "So, I have good new for everyone in today's meeting."},
 {'no': 2,
  'ja_speaker': '岬さん',
  'en_speaker': 'Misaki-san',
  'ja_sentence': 'みなさんに頑張って頂いたおかげで、新しいソフトウェアの売り上げが販売予測を既に超えました。',
  'en_sentence': "Thanks to everyone's hard work, our sales for the new software has already surpassed our sales forecast."},
 {'no': 3,
  'ja_speaker': '岬さん',
  'en_speaker': 'Misaki-san',
  'ja_sentence': 'そこで、すばらしいがんばりをしてくれた皆さんに感謝の印として、何かしたいと考えております。',
  'en_sentence': "Therefore, as a token of appreciation to everyone who put in their great efforts, I'm thinking of doing something for you guys."}]

## La estructura del diccionario que se ve en la muestra anidada se puede pasar a un nuevo DataFrame para visualizarlo como un conjunto nuevo, veamos un ejemplo de como se veria una de las muestras

In [6]:
pd.json_normalize(df['conversation'][12])

,no,ja_speaker,en_speaker,ja_sentence,en_sentence
0,1,岬さん,Misaki-san,さて、本日のミーティングでは、みなさんに良い報告があります。,"So, I have good new for everyone in today's me..."
1,2,岬さん,Misaki-san,みなさんに頑張って頂いたおかげで、新しいソフトウェアの売り上げが販売予測を既に超えました。,"Thanks to everyone's hard work, our sales for ..."
2,3,岬さん,Misaki-san,そこで、すばらしいがんばりをしてくれた皆さんに感謝の印として、何かしたいと考えております。,"Therefore, as a token of appreciation to every..."
3,4,岬さん,Misaki-san,ここより先、向江さんお願いします。,"Mukae-san, I will have you take over from here."
4,5,向江さん,Mukae-san,はい。,Yes.
5,6,向江さん,Mukae-san,皆様、この度は深夜や週末まで詰めて作業して頂きありがとうございました。,"Everyone, thank you all for working diligently..."
6,7,向江さん,Mukae-san,我々経営陣は、皆さんに特別手当の支給または、社内旅行を考えていますが、いかがでしょうか。,Us management team is thinking of providing a ...
7,8,向江さん,Mukae-san,では、開発部の矢崎さん。,"Yazaki-san from the development department, wh..."
8,9,矢崎さん,Yazaki-san,はい。,Yes.
9,10,矢崎さん,Yazaki-san,開発部を代表いたしまして、是非特別手当を支給して頂ければと思います。,As a representative of the development departm...
